### goal of notebook/eventual script: on a per-cell basis, get certain snp stats
- num total snps
- num snps with cov > 1
- num snps in genes
- num snps in peaks
- average/median R2 of SNPs

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os

from scipy.io import mmread, mmwrite

import pybedtools
from pybedtools import BedTool

In [7]:
projdir = '/u/home/t/terencew/project-cluo/igvf/pilot/multiome/'

donors = list(np.loadtxt(f'{projdir}/txt/donors.txt', dtype=str))
samples = list(np.loadtxt(f'{projdir}/txt/samples.txt', dtype=str))
s = samples[0]

In [8]:
indir = f'{projdir}/ambient/ambisim/mux_test/'
exp = '10_20'

In [9]:
cellsnp_dir = f'{indir}/{exp}/demux/varcon_cellsnp/gex/{s}/'

In [10]:
cellsnp_dir

'/u/home/t/terencew/project-cluo/igvf/pilot/multiome//ambient/ambisim/mux_test//10_20/demux/varcon_cellsnp/gex/20220928-IGVF-D0/'

In [11]:
ad = mmread(f'{cellsnp_dir}/cellSNP.tag.AD.mtx').tocsr()
dp = mmread(f'{cellsnp_dir}/cellSNP.tag.DP.mtx').tocsr()

In [60]:
barcodes = pd.read_csv(f'{cellsnp_dir}/cellSNP.samples.tsv', header=None)

In [62]:
barcodes.shape

(8517, 1)

In [12]:
ad, dp

(<504486x8517 sparse matrix of type '<class 'numpy.int64'>'
 	with 2008546 stored elements in Compressed Sparse Row format>,
 <504486x8517 sparse matrix of type '<class 'numpy.int64'>'
 	with 5847055 stored elements in Compressed Sparse Row format>)

In [24]:
vcf = pd.read_csv(f'{cellsnp_dir}/cellSNP.base.vcf.gz', sep='\t', comment='#', header=None)
vcf.shape

(504486, 8)

In [25]:
vcf.head()

,0,1,2,3,4,5,6,7
0,chr1,779825,.,C,G,.,PASS,AD=1;DP=5;OTH=0
1,chr1,785588,.,G,C,.,PASS,AD=0;DP=1;OTH=1
2,chr1,804863,.,T,C,.,PASS,AD=4;DP=11;OTH=0
3,chr1,808040,.,G,A,.,PASS,AD=2;DP=4;OTH=0
4,chr1,830627,.,A,C,.,PASS,AD=1;DP=2;OTH=0


In [26]:
vcf.columns = ['chrom', 'start', 'dot', 'REF', 'ALT', 'dot2', 'PASS', 'counts']
vcf['end'] = vcf['start'] + 1
vcf.index = [f'{x}_{y}' for x,y in zip(vcf['chrom'], vcf['start'])]

In [27]:
vcf

,chrom,start,dot,REF,ALT,dot2,PASS,counts,end
chr1_779825,chr1,779825,.,C,G,.,PASS,AD=1;DP=5;OTH=0,779826
chr1_785588,chr1,785588,.,G,C,.,PASS,AD=0;DP=1;OTH=1,785589
chr1_804863,chr1,804863,.,T,C,.,PASS,AD=4;DP=11;OTH=0,804864
chr1_808040,chr1,808040,.,G,A,.,PASS,AD=2;DP=4;OTH=0,808041
chr1_830627,chr1,830627,.,A,C,.,PASS,AD=1;DP=2;OTH=0,830628
...,...,...,...,...,...,...,...,...,...
chr22_50774447,chr22,50774447,.,A,C,.,PASS,AD=1;DP=4;OTH=0,50774448
chr22_50777832,chr22,50777832,.,G,T,.,PASS,AD=1;DP=6;OTH=0,50777833
chr22_50777878,chr22,50777878,.,G,A,.,PASS,AD=0;DP=4;OTH=1,50777879
chr22_50779128,chr22,50779128,.,C,T,.,PASS,AD=1;DP=2;OTH=0,50779129


In [28]:
snps_bed = vcf[['chrom', 'start', 'end', 'REF', 'ALT', 'counts']]

In [29]:
snps_bed

,chrom,start,end,REF,ALT,counts
chr1_779825,chr1,779825,779826,C,G,AD=1;DP=5;OTH=0
chr1_785588,chr1,785588,785589,G,C,AD=0;DP=1;OTH=1
chr1_804863,chr1,804863,804864,T,C,AD=4;DP=11;OTH=0
chr1_808040,chr1,808040,808041,G,A,AD=2;DP=4;OTH=0
chr1_830627,chr1,830627,830628,A,C,AD=1;DP=2;OTH=0
...,...,...,...,...,...,...
chr22_50774447,chr22,50774447,50774448,A,C,AD=1;DP=4;OTH=0
chr22_50777832,chr22,50777832,50777833,G,T,AD=1;DP=6;OTH=0
chr22_50777878,chr22,50777878,50777879,G,A,AD=0;DP=4;OTH=1
chr22_50779128,chr22,50779128,50779129,C,T,AD=1;DP=2;OTH=0


In [30]:
bc_counts = np.sum(dp, axis=0)
bc_counts.shape

(1, 8517)

In [31]:
bc_counts

matrix([[668, 669, 496, ..., 374, 384, 516]])

In [32]:
tmp_dp = dp > 1
bc_counts_greater = np.sum(tmp_dp, axis=0)
bc_counts_greater.shape

(1, 8517)

In [33]:
bc_counts_greater

matrix([[15, 21, 24, ...,  7,  7,  9]])

### now let's look at num snps in genes/peaks

In [34]:
# gtf_path = '/u/project/cluo/terencew/reference/refdata-cellranger-arc-GRCh38-2020-A-2.0.0/genes/genes.gtf.gz'
# gtf = pd.read_csv(gtf_path, sep='\t', header=None, index_col=None, comment='#')

In [35]:
# gtf.shape

In [36]:
gene_path = '/u/home/t/terencew/project-cluo/reference/hg38/bed/gencode.v39.basic.gene.bed'
gene_bed = pd.read_csv(gene_path, sep='\t', header=None, index_col=None)
gene_bed = gene_bed[[1, 2, 3, 0]]

In [37]:
gene_bed.head()

,1,2,3,0
0,chr1,11869,14409,DDX11L1
1,chr1,14404,29570,WASH7P
2,chr1,17369,17436,MIR6859-1
3,chr1,29554,31109,MIR1302-2HG
4,chr1,30366,30503,MIR1302-2


In [38]:
peaks_path = f'{indir}/{exp}/cr_arc/{s}/outs/atac_peaks.bed'

peaks = pd.read_csv(peaks_path, sep='\t', header=None, index_col=None, comment='#')
peaks.shape

(128166, 3)

In [39]:
snps_bedtool = BedTool.from_dataframe(snps_bed)
genes_bedtool = BedTool.from_dataframe(gene_bed)
peaks_bedtool = BedTool.from_dataframe(peaks)

In [40]:
gene_snps = snps_bedtool.intersect(genes_bedtool).to_dataframe()
gene_snps.index = [f'{x}_{y}' for x,y in zip(gene_snps['chrom'], gene_snps['start'])]
gene_snps = gene_snps[~gene_snps.index.duplicated()]

peak_snps = snps_bedtool.intersect(peaks_bedtool).to_dataframe()
peak_snps.index = [f'{x}_{y}' for x,y in zip(peak_snps['chrom'], peak_snps['start'])]

gene_snps.shape, peak_snps.shape

((503676, 6), (41371, 6))

In [41]:
gene_snps.head()

,chrom,start,end,name,score,strand
chr1_779825,chr1,779825,779826,C,G,AD=1;DP=5;OTH=0
chr1_785588,chr1,785588,785589,G,C,AD=0;DP=1;OTH=1
chr1_804863,chr1,804863,804864,T,C,AD=4;DP=11;OTH=0
chr1_808040,chr1,808040,808041,G,A,AD=2;DP=4;OTH=0
chr1_830627,chr1,830627,830628,A,C,AD=1;DP=2;OTH=0


In [42]:
peak_snps.head()

,chrom,start,end,name,score,strand
chr1_904947,chr1,904947,904948,G,A,AD=2;DP=18;OTH=1
chr1_911484,chr1,911484,911485,G,C,AD=1;DP=2;OTH=0
chr1_913065,chr1,913065,913066,G,A,AD=1;DP=4;OTH=0
chr1_913076,chr1,913076,913077,A,G,AD=2;DP=4;OTH=0
chr1_917063,chr1,917063,917064,A,C,AD=3;DP=13;OTH=0


In [43]:
gene_mask = vcf.index.isin(gene_snps.index)
peak_mask = vcf.index.isin(peak_snps.index)

In [44]:
genes_dp = dp[gene_mask,:]
peaks_dp = dp[peak_mask,:]

In [45]:
peaks_dp

<41371x8517 sparse matrix of type '<class 'numpy.int64'>'
	with 801310 stored elements in Compressed Sparse Row format>

In [46]:
genes_dp

<503848x8517 sparse matrix of type '<class 'numpy.int64'>'
	with 5831926 stored elements in Compressed Sparse Row format>

In [48]:
gene_bc_counts = np.sum(genes_dp, axis=0)
gene_bc_counts.shape

(1, 8517)

In [50]:
gene_bc_counts

matrix([[665, 667, 493, ..., 374, 383, 515]])

In [49]:
tmp_dp = dp > 1
gene_bc_counts_greater = np.sum(tmp_dp, axis=0)
gene_bc_counts_greater.shape

(1, 8517)

In [53]:
bc_counts_greater

matrix([[15, 21, 24, ...,  7,  7,  9]])

In [52]:
gene_bc_counts_greater

matrix([[15, 21, 24, ...,  7,  7,  9]])

### ok great let's make this a package

In [79]:
feature = 'genes'
snp_stats = pd.DataFrame(index = barcodes[0])

In [81]:
snp_stats['n_snps'] = np.ravel(bc_counts)
snp_stats['n_snps > 1'] = np.ravel(bc_counts_greater)
snp_stats[f'n_snps in {feature}'] = np.ravel(gene_bc_counts)
snp_stats[f'n_snps in {feature} > 1'] = np.ravel(gene_bc_counts_greater)

In [82]:
snp_stats

,n_snps,n_snps > 1,n_snps in genes,n_snps in genes > 1
0,,,,
AAACAGCCAAACAACA-1,668,15,665,15
AAACAGCCAAACATAG-1,669,21,667,21
AAACAGCCAAACCCTA-1,496,24,493,24
AAACAGCCAAACCTAT-1,794,28,792,28
AAACAGCCAAACCTTG-1,221,4,221,4
...,...,...,...,...
AACAAGCCACTGACTA-1,629,20,629,20
AACAAGCCACTGGCCA-1,319,5,318,5
AACAAGCCACTGGCTG-1,374,7,374,7
